In [1]:
import os
import pathlib
import subprocess
import sys
import typing

import networkx as nx
import yaml

In [ ]:
# replace the values with the paths on your system
EXECUTABLE_PATH = {
    "vc": "bin/vc",
    "local_ratio_vc": "bin/local_ratio_vc",
}

data_dir = pathlib.Path("data/graphs/")
results_dir = pathlib.Path("results/cvc_and_vc_vs_local_ratio_vc/")
results_dir.mkdir(exist_ok=True)

In [3]:
def graph_in_dir_is_connected(dir: pathlib.Path) -> bool:
    with open(dir / "properties.yaml") as f:
        props = yaml.load(f, yaml.SafeLoader)
        return props["connected"]

In [4]:
def run_on_dir(directory: pathlib.Path) -> None:
    g_str = (directory / "graph.edges").read_text()

    cvc_nodes, cvc_card, vc_nodes, vc_card  = run_vc(g_str)
    local_ratio_vc_nodes, local_ratio_vc_card = run_local_ratio_vc(g_str)

    res_dir = results_dir / directory.name
    res_dir.mkdir(exist_ok=True)

    for contents, filename in [
        (cvc_nodes, "cvc_nodes.txt"),
        (cvc_card, "cvc_cardinality.txt"),
        (vc_nodes, "vc_nodes.txt"),
        (vc_card, "vc_cardinality.txt"),
        (local_ratio_vc_nodes, "local_ratio_vc_nodes.txt"),
        (local_ratio_vc_card, "local_ratio_vc_cardinality.txt"),
    ]:
        with open(res_dir / filename, "w") as f:
            _ = f.write(contents)


def run_vc(g_str: str) -> tuple[str, str, str, str]:
    out = run_executable_on_graph(EXECUTABLE_PATH["vc"], g_str).split("\n", maxsplit=1)
    assert len(out) == 2, f"error: output of vc is expected to have two lines"
    return parse_output(out[0]) + parse_output(out[1])


def run_local_ratio_vc(g_str: str) -> tuple[str, str]:
    return parse_output(run_executable_on_graph(EXECUTABLE_PATH["local_ratio_vc"], g_str))


def run_executable_on_graph(exe_path: pathlib.Path | os.PathLike[typing.Any] | str, g_str: str) -> str:
    p = pathlib.Path(exe_path)
    assert p.is_file(), f"error: {p} is not a file or does not exist."

    proc = subprocess.run(
        str(exe_path),
        input=g_str,
        capture_output=True,
        text=True,
    )

    assert proc.returncode == 0, f'error: subprecess process returned {proc.returncode}; stderr= "{proc.stderr}"'
    return proc.stdout


def parse_output(text: str) -> tuple[str, str]:
    nodes, cardinality = text.split(" => ", maxsplit=1)
    nodes = nodes.replace(", ", "\n") + "\n"
    cardinality = cardinality.strip()
    return nodes, cardinality

In [6]:
for directory in data_dir.iterdir():
    print(f"{directory.name} => ", end="")

    if not graph_in_dir_is_connected(directory):
        print("skipeed (not connected)")
        continue

    run_on_dir(directory)

    print(f"results saved to {results_dir.name}/{directory.name}")

print("Done.")

bio-WormNet-v3_lcc => results saved to cvc_and_vc_vs_local_ratio_vc/bio-WormNet-v3_lcc
CL-100K-1d8-L9_lcc => results saved to cvc_and_vc_vs_local_ratio_vc/CL-100K-1d8-L9_lcc
jan99jac020 => results saved to cvc_and_vc_vs_local_ratio_vc/jan99jac020
bcsstk17_lcc => results saved to cvc_and_vc_vs_local_ratio_vc/bcsstk17_lcc
RFdevice => results saved to cvc_and_vc_vs_local_ratio_vc/RFdevice
mark3jac120sc => results saved to cvc_and_vc_vs_local_ratio_vc/mark3jac120sc
aft02 => results saved to cvc_and_vc_vs_local_ratio_vc/aft02
cage11 => results saved to cvc_and_vc_vs_local_ratio_vc/cage11
chem_master1 => results saved to cvc_and_vc_vs_local_ratio_vc/chem_master1
brack2 => results saved to cvc_and_vc_vs_local_ratio_vc/brack2
bayer01 => results saved to cvc_and_vc_vs_local_ratio_vc/bayer01
pkustk01 => results saved to cvc_and_vc_vs_local_ratio_vc/pkustk01
COIL-DEL_lcc => results saved to cvc_and_vc_vs_local_ratio_vc/COIL-DEL_lcc
ibm_matrix_2 => results saved to cvc_and_vc_vs_local_ratio_vc/ibm